In [ ]:
!pip install transformers  #安裝必要套件
!pip install datasets

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM   #定義預訓練模型 (PLM) 和special token
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

plm = "EleutherAI/pythia-160m"

bos = '<|endoftext|>'
eos = '<|END|>'
pad = '<|pad|>'
sep ='\n\n####\n\n'

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad, 'sep_token': sep}

tokenizer = AutoTokenizer.from_pretrained(plm, revision="step3000")
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
tokenizer.padding_side = 'left'

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset, Features, Value   #載入分割好的文本
dataset = load_dataset("csv", data_files="Train.tsv", delimiter='\t',
                       features = Features({
                              'fid': Value('string'), 'idx': Value('int64'),
                              'content': Value('string'), 'label': Value('string')}),
                       column_names=['fid', 'idx', 'content', 'label'], keep_default_na=False)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
import torch  #資料集分割
sub_datasets = torch.utils.data.random_split(dataset['train'], [89822, 0])#85736

In [ ]:
PAD_IDX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token) #PAD
IGNORED_PAD_IDX = -100
PAD_IDX

In [ ]:
from torch.utils.data import DataLoader   #進行分詞索引和創建DataLoader
import torch

train_data = list(sub_datasets[0])
# val_data = list(sub_datasets[1])

def collate_batch(batch):
    texts = [f"{bos} {data['content']} {sep}"+ data['label'].replace('\\n','\n')+f" {eos}" for data in list(batch)] # 範例 prompt
    encoded_seq = tokenizer(texts, padding=True)

    indexed_tks = torch.tensor(encoded_seq['input_ids'])
    attention_mask = torch.tensor(encoded_seq['attention_mask'])
    encoded_label = torch.tensor(encoded_seq['input_ids'])
    encoded_label[encoded_label == tokenizer.pad_token_id] = IGNORED_PAD_IDX

    return indexed_tks, encoded_label, attention_mask

train_dataloader = DataLoader(train_data, batch_size=2, shuffle=False, collate_fn=collate_batch)
titer = iter(train_dataloader)
tks, labels, masks= next(titer)

# val_dataloader = DataLoader(val_data, batch_size=2, shuffle=False, collate_fn=collate_batch)
# val_titer = iter(val_dataloader)
# val_tks, val_labels, val_masks= next(val_titer)

# print(tks.shape)
# next(iter(titer))

# print(val_tks.shape)
# next(iter(val_titer))


In [ ]:
import random  #批次取樣器、打亂
BATCH_SIZE = 5

class BatchSampler():
    def __init__(self, data, batch_size):
        self.pooled_indices = []
        self.data = data
        self.batch_size = batch_size
        self.len = len(list(data))
    def __iter__(self):
        self.pooled_indices = []
        indices = [(index, len(data["content"])) for index, data in enumerate(self.data)]
        random.shuffle(indices)
        for i in range(0, len(indices), BATCH_SIZE * 100):
            self.pooled_indices.extend(sorted(indices[i:i + BATCH_SIZE * 100], key=lambda x: x[1], reverse=True))
        self.pooled_indices = [x[0] for x in self.pooled_indices]

        for i in range(0, len(self.pooled_indices), BATCH_SIZE):
            yield self.pooled_indices[i:i + BATCH_SIZE]
    def __len__(self):
        return (self.len + self.batch_size - 1) // self.batch_size

bucket_train_dataloader = DataLoader(train_data, batch_sampler=BatchSampler(train_data, BATCH_SIZE),
                                     collate_fn=collate_batch, pin_memory=True)

# bucket_val_dataloader = DataLoader(val_data, batch_sampler=BatchSampler(val_data, BATCH_SIZE),
#                                      collate_fn=collate_batch, pin_memory=True)


In [ ]:
from transformers import AutoConfig    #模型配置
config = AutoConfig.from_pretrained(plm,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    pad_token_id=tokenizer.pad_token_id,
                                    sep_token_id=tokenizer.sep_token_id,
                                    output_hidden_states=False)

model = AutoModelForCausalLM.from_pretrained(plm, revision="step3000", config=config)
model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   #使用GPU
model.to(device)

In [ ]:
# import torch
# from tqdm import tqdm#, tqdm_notebook
# from torch.nn import functional as F

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# def sample_text(model, tokenizer, seed, n_words=20):
#     model = model.to(device)
#     model.eval()
#     text = tokenizer.encode(seed)
#     inputs, past_key_values = torch.tensor([text]), None
#     with torch.no_grad():
#         for _ in tqdm(range(n_words)):
#             out = model(inputs.to(device), past_key_values=past_key_values)
#             logits = out.logits
#             past_key_values = out.past_key_values
#             log_probs = F.softmax(logits[:, -1], dim=-1)
#             inputs = torch.multinomial(log_probs, 1)
#             text.append(inputs.item())
#             if tokenizer.decode(inputs.item()) == eos:
#                 break


#     return tokenizer.decode(text)

# sample_text(model, tokenizer, seed=f"{bos} DR AADLAND ABRAHAM {sep}")

100%|██████████| 20/20 [00:12<00:00,  1.62it/s]


'<|endoftext|> DR AADLAND ABRAHAM \n\n####\n\n! With a Body Examination! I made a Field at Fedaneman JKL Monday, Wednesday,'

In [ ]:
from transformers import get_linear_schedule_with_warmup   #優化器和學習率調度器
from torch.optim import AdamW

EPOCHS = 10 # 訓練次數
optimizer = AdamW(model.parameters(),lr=7e-5)

steps = len(bucket_train_dataloader)
total_steps = steps * EPOCHS
print(steps, total_steps)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=total_steps*0.1,
    num_training_steps=total_steps
)

model.resize_token_embeddings(len(tokenizer))
model.to(device)
print(f'Total numbers of steps: {total_steps}')
model


17965 179650
Total numbers of steps: 179650


GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50280, 768)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
          (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
          

In [ ]:
from tqdm import tqdm,trange   #訓練迴圈

global_step = 0
total_loss = 0

model.train()
for epoch in trange(EPOCHS, desc="Epoch"):
    model.train()
    total_loss = 0

    predictions , true_labels = [], []

    for step, (seqs, labels, masks) in enumerate(bucket_train_dataloader):
        seqs = seqs.to(device)
        labels = labels.to(device)
        masks = masks.to(device)
        model.zero_grad()
        outputs = model(seqs, labels=labels, attention_mask=masks)
        logits = outputs.logits
        loss = outputs.loss
        loss = loss.mean()
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
    avg_train_loss = total_loss / len(bucket_train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))
    # 儲存模型和分詞器
    model_save_path = f"/content/drive/MyDrive/models/epoch_{epoch + 1}"
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Average train loss: 1.523700798088682


Epoch:  10%|█         | 1/10 [26:31<3:58:47, 1591.92s/it]

Average train loss: 1.1826074210823727


Epoch:  20%|██        | 2/10 [53:09<3:32:41, 1595.14s/it]

...

Epoch:  80%|████████  | 8/10 [3:32:51<53:11, 1595.88s/it]  

Average train loss: 0.5231661353076762


Epoch:  90%|█████████ | 9/10 [3:59:31<26:37, 1597.16s/it]

Average train loss: 0.4984179870505171


Epoch: 100%|██████████| 10/10 [4:25:57<00:00, 1595.79s/it]
